In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib

In [3]:
# Data Files Locations

file_loc_train = "data_miccai/train"
file_loc_test = "data_miccai/test"
file_loc_val = "data_miccai/val"

In [4]:
# list of all the folders in the train location 
list_of_dir  = os.listdir(file_loc_train)

print("Total folders :", len(list_of_dir))

Total folders : 1000


In [5]:
# This function will find the average of the values in a column.
# It will skip the cell with value zero "0".

def blood_data_avg(pd_data):
    row,column = pd_data.shape
    
    arr_val = [0]*column
    
    for i in range(column):
        sm = 0
        cnt = 0
        for j in range(row):
            if pd_data.iloc[j,i] != 0:
                sm += pd_data.iloc[j,i]
                cnt+=1
        arr_val[i] = sm/cnt
    return arr_val


def radiology_data_avg(pd_data):
    row,column = pd_data.shape
    
    arr_val = [0]*column
    
    for i in range(column):
        sm = 0
        cnt=0
        if i==0:
            for j in range(row):
                if pd_data.iloc[j,i] != 0:
                    sm += pd_data.iloc[j,i]
                    cnt+=1
            arr_val[i] = sm/(cnt*1.0)
        else:
            for j in range(row):
                if(pd_data.iloc[j,i] == True):
                    sm+=1
                cnt+=1
            if(sm>=(cnt/2.0)):
                arr_val[i]=1
    return arr_val

def temperature_data_avg(pd_data):
    row,column = pd_data.shape
    
    arr_val = [0]*column
    
    for i in range(column):
        sm = 0
        cnt = 0
        for j in range(row):
            if pd_data.iloc[j,i] != 0:
                sm += pd_data.iloc[j,i]
                cnt+=1
        arr_val[i] = sm/cnt
    return arr_val

def outcome_data_avg(pd_data):
    row,column = pd_data.shape
    
    output = 0
    for i in range(1,column):
        if pd_data.iloc[0,i] == 1:
            output += 2**(i-1)
    return output
    

In [6]:
output_combined_file_loc = "output/data_blood_radiology_temperature_train.csv"
Y_file_loc = "output/Y_blood_radiology_temperature_train.csv"

output_combined = []
Y = []

for i in list_of_dir:
    cd_list_of_dir = file_loc_train+"/"+i
    
    file_read_blood = pd.read_csv(cd_list_of_dir+"/data_bloodvalues.csv", sep=",")
    file_read_radiology = pd.read_csv(cd_list_of_dir+"/data_radiology.csv", sep=",")
    file_read_temperature = pd.read_csv(cd_list_of_dir+"/data_temperature.csv", sep=",")
    
    file_read_outcome = pd.read_csv(cd_list_of_dir+"/outcome.csv", sep=",")
    Y.append(outcome_data_avg(file_read_outcome))
    
    output_combined.append(blood_data_avg(file_read_blood)[1:] + radiology_data_avg(file_read_radiology)[1:] + temperature_data_avg(file_read_temperature)[1:])

In [7]:
output_data_combined_pd = pd.DataFrame(output_combined)
output_data_combined_pd.to_csv(output_combined_file_loc, sep=',', index=False)

Y_pd = pd.DataFrame(Y)
Y_pd.to_csv(Y_file_loc, sep=',', index=False)

In [8]:
# Training XGBoost
X_train = output_data_combined_pd
y_train = Y_pd

In [9]:
# fit model no training data
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=50, max_depth=10)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [10]:
# list of all the folders in the train location 
list_of_dir  = os.listdir(file_loc_test)

print("Total folders :", len(list_of_dir))

Total folders : 250


In [11]:
output_combined_file_loc = "output/data_blood_radiology_temperature_test.csv"
Y_file_loc = "output/Y_blood_radiology_temperature_test.csv"

output_combined = []
Y = []

for i in list_of_dir:
    cd_list_of_dir = file_loc_test+"/"+i
    
    file_read_blood = pd.read_csv(cd_list_of_dir+"/data_bloodvalues.csv", sep=",")
    file_read_radiology = pd.read_csv(cd_list_of_dir+"/data_radiology.csv", sep=",")
    file_read_temperature = pd.read_csv(cd_list_of_dir+"/data_temperature.csv", sep=",")
    
    file_read_outcome = pd.read_csv(cd_list_of_dir+"/outcome.csv", sep=",")
    Y.append(outcome_data_avg(file_read_outcome))
    
    output_combined.append(blood_data_avg(file_read_blood)[1:] + radiology_data_avg(file_read_radiology)[1:] + temperature_data_avg(file_read_temperature)[1:])

In [12]:
output_data_combined_pd = pd.DataFrame(output_combined)
output_data_combined_pd.to_csv(output_combined_file_loc, sep=',', index=False)

Y_pd = pd.DataFrame(Y)
Y_pd.to_csv(Y_file_loc, sep=',', index=False)

In [13]:
# Training XGBoost
X_test = output_data_combined_pd
y_test = Y_pd

In [14]:
res = model.predict(X_test)

In [15]:
j = 0
cnt = 0
for i in y_test[0]:
    if i == res[j]:
        cnt+=1
    j+=1

print("accuracy:",cnt/len(res))

accuracy: 0.564
